In [1]:
from matplotlib import pyplot as plt
from hddm.simulators import simulator
import hddm
import numpy as np
import pandas as pd
import scipy.sparse as sp
import gc

import sys
sys.path.append("../Codes")
from utils import *
from chain import *
from Hall1997 import *


In [2]:
hddm.model_config.model_config["angle"]


{'doc': 'Model formulation is described in the documentation under LAN Extension.\nMeant for use with the extension.',
 'params': ['v', 'a', 'z', 't', 'theta'],
 'params_trans': [0, 0, 1, 0, 0],
 'params_std_upper': [1.5, 1.0, None, 1.0, 1.0],
 'param_bounds': [[-3.0, 0.3, 0.1, 0.001, -0.1], [3.0, 3.0, 0.9, 2.0, 1.3]],
 'boundary': <function ssms.basic_simulators.boundary_functions.angle(t=1, theta=1)>,
 'params_default': [0.0, 1.0, 0.5, 0.001, 0.1],
 'hddm_include': ['v', 'a', 't', 'z', 'theta'],
 'choices': [-1, 1],
 'slice_widths': {'v': 1.5,
  'v_std': 1,
  'a': 1,
  'a_std': 1,
  'z': 0.1,
  'z_trans': 0.2,
  't': 0.01,
  't_std': 0.15,
  'theta': 0.1,
  'theta_std': 0.2}}

### test 1: Set $\theta=0$

Note that the space considered in angle model and LAN is $[-a,a]$, while that in DDM and wfpt is $[0, a]$.

Also the angle model and LAN implements the non-decision time.

#### plot the first passage time density

In [ ]:
# parameters
mu = 1.0
a = 1.5
z = 0.5
t_nd = 0.5
theta = 0
out = simulator(model="angle", theta=[mu, a, z, t_nd, theta], n_samples=10000)


In [ ]:
rts, choices, metadata = out
print("rts: " + str(rts.shape))
print("choices: " + str(choices.shape))
print("metadata: ")
for key in metadata:
    print(key, end=": ")
    print(metadata[key])


In [ ]:
# Get data into format accepted by hddm package (below)
data = pd.DataFrame(
    np.stack([np.squeeze(out[0]), np.squeeze(out[1])]).T, columns=["rt", "response"]
)
data["subject"] = 0
data


In [ ]:
# Get the model
lan_angle = hddm.network_inspectors.get_torch_mlp(model="angle")


In [ ]:
# prob1 stands for wfpt
# prob2 stands for MC_Angle
# prob3 stands for MC_DDM
# prob4 stands for LAN
# prob5 stands for Hall1997

prob1_list = []
prob2_list = []
prob3_list = []
prob5_list = []

mc2 = MC_Angle(mu=mu, a=a, z=z, t_nd=t_nd, theta=theta, dt=0.001, Nx=80)
mc3 = MC_DDM(mu=mu, sigma=1, a=2 * a, z=z, dt=0.001, Nx=80, verbose=False)

T_list = np.linspace(0.5+0.001, 8, 100)
rT_list = np.hstack((T_list, T_list)).reshape(-1, 1)
choice_list = np.hstack((np.ones(T_list.shape), -np.ones(T_list.shape))).reshape(-1, 1)
para_matrix = np.tile(np.array([mu, a, z, t_nd, theta]), (200, 1))
network_input = np.hstack((para_matrix, rT_list, choice_list)).astype(np.float32)
prob4_list = np.exp(lan_angle(network_input))

print("upper:")
for T in T_list:
    print("{:.4f}".format(T), end=" ")
    prob1 = wfpt(T - t_nd, -mu, 2 * a, 1-z, 1e-10)
    prob2 = mc2.ExitProb_dp(T=T, s=a) / mc2.dt
    prob3 = mc3.ExitProb_dp1(T=T - t_nd, s=2*a) / mc3.dt
    prob5 = density_horiz(T-t_nd, mu, a, theta, trunc_num=40, bdy="upper")
    prob1_list.append(prob1)
    prob2_list.append(prob2)
    prob3_list.append(prob3)
    prob5_list.append(prob5)
    gc.collect()
print()
print("lower:")
for T in T_list:
    print("{:.4f}".format(-T), end=" ")
    prob1 = wfpt(T - t_nd, mu, 2 * a, z, 1e-10)
    prob2 = mc2.ExitProb_dp(T=T, s=-a) / mc2.dt
    prob3 = mc3.ExitProb_dp1(T=T - t_nd, s=0) / mc3.dt
    prob5 = density_horiz(T-t_nd, mu, a, theta, trunc_num=40, bdy="lower")
    prob1_list.append(prob1)
    prob2_list.append(prob2)
    prob3_list.append(prob3)
    prob5_list.append(prob5)
    gc.collect()
    


In [ ]:
sort_idx = np.argsort(network_input[:, -2] * network_input[:, -1])
signedT_list = (network_input[:, -2] * network_input[:, -1])[sort_idx]

prob1_array = np.array(prob1_list)[sort_idx]
prob2_array = np.array(prob2_list)[sort_idx]
prob3_array = np.array(prob3_list)[sort_idx]
prob4_array = np.array(prob4_list)[sort_idx]
prob5_array = np.array(prob5_list)[sort_idx]

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3.6), dpi=150)
ax.plot(signedT_list, prob1_array, label='WFPT, Navarro and Fuss (2009)', lw=0.8, c='g')
ax.plot(signedT_list, prob2_array, label='MC_Angle', lw=0.8, c='r')
ax.plot(signedT_list, prob3_array, label='MC_ddm', lw=0.8, c='purple')
ax.plot(signedT_list, prob4_array, label='LAN', lw=0.8, c='b')
ax.plot(signedT_list, prob5_array, label='Hall (1997)', lw=0.8, c='y')

ax.hist(
    np.squeeze(out[0]) * np.squeeze(out[1]), histtype="step", bins=40, color="black", density=True
)
ax.set_xlabel("reaction times")
ax.set_ylabel("density")
ax.legend(fontsize=6)
plt.show()

### test 2: For the Angle model with $\theta\neq0$

In [2]:
mu = 1
a = 1.5
z = 0.5
t_nd = 0.5
theta = 0.3
out = simulator(model="angle", theta=[mu, a, z, t_nd, theta], n_samples=10000)

In [ ]:
prob2_list = []
prob5_list = []

mc2 = MC_Angle(mu=mu, a=a, z=z, t_nd=t_nd, theta=theta, dt=0.001, Nx=80) # dt=0.0002, Nx=200 for a dense approx

T_list = np.linspace(0.5+0.001, 8, 100)
rT_list = np.hstack((T_list, T_list)).reshape(-1, 1)
choice_list = np.hstack((np.ones(T_list.shape), -np.ones(T_list.shape))).reshape(-1, 1)
para_matrix = np.tile(np.array([mu, a, z, t_nd, theta]), (200, 1))
network_input = np.hstack((para_matrix, rT_list, choice_list)).astype(np.float32)
prob4_list = np.exp(lan_angle(network_input))


print("upper:")

flag = True
for T in T_list:
    print("{:.4f}".format(T), end=" ")
    if flag:
        prob5 = density_horiz(T-t_nd, mu, a, theta, trunc_num=10, bdy="upper")
        if prob5 < 0:
            flag = False
            print()
            print("Hall1997 method breaks at T-t_nd=%.4f" %(T-t_nd))
            prob5_list.append(-1)
        else:
            prob5_list.append(prob5)
    else:
        prob5_list.append(-1)
    prob2 = mc2.ExitProb_dp(T=T, s=mc2.upper_bdy(T-mc2.t_nd)) / mc2.dt
    prob2_list.append(prob2)
    gc.collect()

print()
print("lower:")

flag = True
for T in T_list:
    print("{:.4f}".format(-T), end=" ")
    if flag:
        prob5 = density_horiz(T-t_nd, mu, a, theta, trunc_num=10, bdy="lower")
        if prob5 < 0:
            flag = False
            print()
            print("Hall1997 method breaks at T-t_nd=%.4f" %(T-t_nd))
            prob5_list.append(-1)
        else:
            prob5_list.append(prob5)
    else:
        prob5_list.append(-1)
    prob2 = mc2.ExitProb_dp(T=T, s=mc2.lower_bdy(T-mc2.t_nd)) / mc2.dt
    prob2_list.append(prob2)
    gc.collect()
    


In [ ]:
sort_idx = np.argsort(network_input[:, -2] * network_input[:, -1])
signedT_list = (network_input[:, -2] * network_input[:, -1])[sort_idx]

prob2_array = np.array(prob2_list)[sort_idx]
prob4_array = np.array(prob4_list)[sort_idx]
prob5_array = np.array(prob5_list)[sort_idx]

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3.6), dpi=150)
ax.plot(signedT_list, prob2_array, label='MC_Angle', lw=0.8, c='r')
ax.plot(signedT_list, prob4_array, label='LAN', lw=0.8, c='b')
ax.plot(signedT_list[prob5_array>0], prob5_array[prob5_array>0], label='Hall (1997)', lw=0.8, c='y')

ax.hist(
    np.squeeze(out[0]) * np.squeeze(out[1]), histtype="step", bins=40, color="black", density=True
)
ax.set_xlabel("reaction times")
ax.set_ylabel("density")
ax.legend(fontsize=6)
plt.show()

In [3]:
density_horiz(4., mu, a, theta, trunc_num=10, bdy="upper", debug=True)

even_term	   odd_term		term		result
1.35121639e-02	-2.58472083e-02	-1.23350444e-02	-1.23350444e-02
1.75144845e-02	-6.35664334e-03	1.11578412e-02	-1.17720324e-03
1.35095924e-03	-1.74032291e-04	1.17692695e-03	-2.76294007e-07
1.38224620e-05	-6.83448340e-07	1.31390136e-05	1.28627196e-05
2.11642664e-08	-4.12111988e-10	2.07521545e-08	1.28834718e-05
5.06006124e-12	-3.92560528e-14	5.02080519e-12	1.28834768e-05
1.92720804e-16	-5.99417801e-19	1.92121387e-16	1.28834768e-05
1.18224812e-21	-1.47973715e-24	1.18076839e-21	1.28834768e-05
1.17602500e-27	-5.93769480e-31	1.17543123e-27	1.28834768e-05
1.90531263e-34	-3.88697109e-38	1.90492394e-34	1.28834768e-05


1.2883476808630463e-05

In [8]:
density_vertical(0, mu, a, theta, 4, trunc_num=10, debug=True)

0.010878304085912983
-0.02116157354294213
-0.005651240983833719
-0.00017833345806486572
-8.493828594879026e-07
-6.474702550996626e-10
-8.05731596976229e-14
-1.6492008648478324e-18
-5.5687370955190705e-24
-3.1056842310976825e-30


0.010881789327255982